# Photometric redshifts for CFIS in DEEP2+3 field
Run random forest.

In [1]:
import os
import sys

In [2]:
py_v = '3.7'
path = f"{os.environ['CONDA_PREFIX']}/../shapepipe/lib/python{py_v}/site-packages/shapepipe-0.0.2-py{py_v}.egg"
sys.path.append(path)
from shapepipe.utilities.canfar import download, dir_list
from shapepipe.utilities.file_system import mkdir

KeyError: 'CONDA_PREFIX'

In [3]:
import params_ps3pi_cfis as params

In [ ]:
for dir_name in [params.catalog_path, params.matched_path, params.unmatched_path]:
    mkdir(dir_name)

In [4]:
survey = 'ps3pi_cfis'

## Pre-processing

### Clean-up previous run(s)

In [ ]:
! python photoz.py --survey $survey --clean True --input params_$survey

## Download input catalogs

In [ ]:
from io import StringIO
from contextlib import redirect_stdout
from shapepipe.utilities.canfar import download, dir_list, vosHandler

def Ddir_list(path, verbose=False):
    """list

    List content of path on vos

    Parameters
    ----------
    path : string
        path on vos, starts with 'vos:cfis/...'
    verbose : bool, optional, default=False
        verbose output if True

    Raises
    ------
    HTTPError, KeyError

    Returns
    -------
    vls_out : array of string
        file or directory at path
    """

    cmd = 'vls'
    sys.argv = [cmd, path]
    vls = vosHandler(cmd)

    if verbose:
        print('Getting vos directory content from vls...')

    f = StringIO()

    try:
        with redirect_stdout(f):
            vls()
    except:
        print('Error during vls command')
        raise

    vls_out = f.getvalue()
    return vls_out.split('\n')

In [ ]:
vos_dir = 'vos:cfis/cosmostat/catalogues/multi_band_matched/W3_deep23'

# ...
cat_name = 'R_CFHT_vs_CFIS.csv'
cat_path = f'{params.catalog_path}/{cat_name}'
download(f'{vos_dir}/{cat_name}', cat_path, verbose=True)

# Matched and unmatched catalogues
vos_dir_matched = f'{vos_dir}/ps3pi_cfis/matched'
vos_dir_unmatched = f'{vos_dir}/ps3pi_cfis/unmatched'

# If dir_list not found, replace with Ddir_list
f_list = dir_list(vos_dir_matched)
for f in f_list:
    download(f'{vos_dir_matched}/{f}', f'{params.matched_path}/{f}', verbose=True)
f_list = dir_list(vos_dir_unmatched)
for f in f_list:
    download(f'{vos_dir_unmatched}/{f}', f'{params.unmatched_path}/{f}', verbose=True)

### Create invidivual tile catalogs

In [ ]:
! python photoz.py --survey $survey --make True --input params_$survey

### Join individual tile catalogs

In [ ]:
! python photoz.py --survey $survey --join True --input params_$survey

### Create plots of the redshift distributions
Figures are saves as `pdf` in `output/ps3pi_cfis/CFIS_matched_deep_2_3_catalog_R/figures`.

In [ ]:
! python photoz.py --survey $survey --generate_plots True --input params_$survey

## Machine-learning

### Train random forest

In [17]:
! python photoz.py --survey $survey --learning True --preprocess drop --algorithm RF --optimize HyperOpt --input params_$survey

MKDEBUG  ps3pi_cfis ['R'] /automnt/n17data/mkilbing/astro/repositories/github/shapepipe_photoz/output/ps3pi_cfis/CFIS_matched_deep_2_3_catalog_R/files/CFIS_matched_deep_2_3_catalog_R.csv CFIS_matched_deep_2_3_catalog_R /automnt/n17data/mkilbing/astro/repositories/github/shapepipe_photoz/ 4 drop 1


### Stand-alone plots

In [6]:
from functions import *

In [40]:
path_to_files =  params.output_path + 'output/' + survey  + '/' + params.output_name + '/files/'
path_to_csv =  f'{path_to_files}/{params.output_name}.csv'
ML = LearningAlgorithms(survey=survey, bands=params.bands, path_to_csv=path_to_csv,
                        output_name=params.output_name, output_path=params.output_path,
                        cv=params.cv, preprocessing='drop', n_jobs=1)


method = 'RF_Opt'
yy_path = f'{path_to_files}/ML/{method}_prediction_{params.output_name}.csv'
df = pd.read_csv(yy_path)
z_max = 1.3
ML.plot_zphot_zspec(df.y_pred, df.y_test, 'RF', z_max, y_lim=z_max, xy_min=0)

TypeError: plot_zphot_zspec() got an unexpected keyword argument 'y_lim'

In [34]:
df

,y_pred,y_test
0,0.807483,3.739988
1,0.506931,0.280190
2,0.938688,0.592432
3,1.113467,0.835395
4,0.808815,0.629274
...,...,...
550,0.402573,0.422956
551,0.370285,0.533643
552,0.944577,1.217829
553,0.293669,0.199628


In [25]:
path_to_csv

'/automnt/n17data/mkilbing/astro/repositories/github/shapepipe_photoz/output/ps3pi_cfis/CFIS_matched_deep_2_3_catalog_R/files/CFIS_matched_deep_2_3_catalog_R.csv'